In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [23]:
import nltk

In [18]:
dataset = pd.read_csv('./data/Kaggle/train.csv',delimiter=',',\
                           names=['id','keyword','location', 'text','target'])
dataset.head()

,id,keyword,location,text,target
0,id,keyword,location,text,target
1,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
2,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
3,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
4,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1


In [19]:
# Drop Id, Keyword, Location
dataset = dataset.drop(labels=['id', 'keyword','location'], axis=1)
dataset.head()

,text,target
0,text,target
1,Our Deeds are the Reason of this #earthquake M...,1
2,Forest fire near La Ronge Sask. Canada,1
3,All residents asked to 'shelter in place' are ...,1
4,"13,000 people receive #wildfires evacuation or...",1


In [20]:
### Clean data
# Drop first row
dataset = dataset.drop(index=0)
dataset.head()

,text,target
1,Our Deeds are the Reason of this #earthquake M...,1
2,Forest fire near La Ronge Sask. Canada,1
3,All residents asked to 'shelter in place' are ...,1
4,"13,000 people receive #wildfires evacuation or...",1
5,Just got sent this photo from Ruby #Alaska as ...,1


In [22]:
# Check if there are target entries without 0 or 1
dataset_target = dataset['target']
print(dataset_target.value_counts(dropna=False))

0    4342
1    3271
Name: target, dtype: int64


In [30]:
# Check how many words can be tokenized by BERT
# Remove punctuation, split by spaces
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
dataset['text'].apply(tokenizer.tokenize)


#tokenizer.tokenize('Coooool, and 1 sndj-dkdsn :)')

1       [Our, Deeds, are, the, Reason, of, this, earth...
2           [Forest, fire, near, La, Ronge, Sask, Canada]
3       [All, residents, asked, to, shelter, in, place...
4       [13, 000, people, receive, wildfires, evacuati...
5       [Just, got, sent, this, photo, from, Ruby, Ala...
                              ...                        
7609    [Two, giant, cranes, holding, a, bridge, colla...
7610    [aria_ahrary, TheTawniest, The, out, of, contr...
7611    [M1, 94, 01, 04, UTC, 5km, S, of, Volcano, Haw...
7612    [Police, investigating, after, an, e, bike, co...
7613    [The, Latest, More, Homes, Razed, by, Northern...
Name: text, Length: 7613, dtype: object

In [31]:
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
dataset['text'].apply(tt.tokenize)

1       [Our, Deeds, are, the, Reason, of, this, #eart...
2        [Forest, fire, near, La, Ronge, Sask, ., Canada]
3       [All, residents, asked, to, ', shelter, in, pl...
4       [13,000, people, receive, #wildfires, evacuati...
5       [Just, got, sent, this, photo, from, Ruby, #Al...
                              ...                        
7609    [Two, giant, cranes, holding, a, bridge, colla...
7610    [@aria_ahrary, @TheTawniest, The, out, of, con...
7611    [M1, ., 94, [, 01:04, UTC, ], ?, 5km, S, of, V...
7612    [Police, investigating, after, an, e-bike, col...
7613    [The, Latest, :, More, Homes, Razed, by, North...
Name: text, Length: 7613, dtype: object